In [80]:
import pandas as pd

# 确保所有列都能显示出来
pd.set_option('display.max_columns', 500)

# 确保列宽足够，不会把长字符串（比如 Method 名）截断
pd.set_option('display.max_colwidth', 100)

# 确保表格的总宽度足够，不会换行显示
pd.set_option('display.width', 1000)

In [81]:
TASK_METRIC = {
    "cola": ["eval_matthews_correlation"],
    "mnli": ["matched_accuracy", "mismatched_accuracy"],
    "mrpc": ["eval_accuracy", "eval_f1"],
    "qnli": ["eval_accuracy"],
    "qqp": ["eval_accuracy", "eval_f1"],
    "rte": ["eval_accuracy"],
    "sst2": ["eval_accuracy"],
    "stsb": ["eval_pearson", "eval_spearman"],
    "wnli": ["eval_accuracy"],
}

METRIC_NAME_MAP = {
    'eval_matthews_correlation': 'Mcc',
    'matched_accuracy': 'm',
    'mismatched_accuracy': 'mm',
    'eval_accuracy': 'Acc',
    'eval_f1': 'F1',
    'eval_pearson': 'Corr_p',
    'eval_spearman': 'Corr_s',
}

TASK_NAME_MAP = {
    'mnli': 'MNLI',
    'sst2': 'SST-2',
    'cola': 'CoLA',
    'qqp': 'QQP',
    'qnli': 'QNLI',
    'rte': 'RTE',
    'mrpc': 'MRPC',
    'stsb': 'STS-B',
}

In [82]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from dictor import dictor
import seaborn as sns
import matplotlib.pyplot as plt

def extract_experiment_data(json_file):
    variant = Path(json_file).relative_to('./results').parts[0]

    with open(json_file, 'r') as f:
        data = json.load(f)

    data['variant'] = variant
    # with open(json_file, 'w') as f:
    #     json.dump(data, f, indent=4)

    # Extract metadata
    model_family = dictor(data, 'args.model_family')
    peft_method = dictor(data, 'args.peft')
    task = dictor(data, 'args.task')

    eval_runtime = data.get('eval_runtime', -1)

    # Get training-specific metrics
    trainable_params = dictor(data, 'train.trainable_params_count', -1)
    train_runtime = dictor(data, 'train.train_time', -1)

    # Calculate Average GPU Memory (Allocated)
    memory_list = dictor(data, 'train.memory_allocated', [])
    avg_memory = np.mean(memory_list) if memory_list else -1

    rank = dictor(data, 'args.rank')
    if 'mrlora' in peft_method:
        rank = 2*rank - 1 # r = 2*R - 1
        
    # Get metrics
    # Some tasks use eval_accuracy, others eval_matthews_correlation
    for key in TASK_METRIC[task]:
        if key in data:
            accuracy = data[key]
            yield {
                "family": model_family,
                "peft": peft_method,
                "task": task,
                "variant": variant,
                "value": round(accuracy, 4),
                "metric": key,
                "params": round(trainable_params, 4),
                "traintime": round(train_runtime, 2),
                "evaltime": round(eval_runtime, 2),
                "gpumem": round(avg_memory, 2),
                "rank": rank, # total rank.
            }


def aggregate_experiment_results(root_dir):
    """
    Finds all .json files under a directory recursively, extracts data,
    and concatenates them into one large DataFrame.
    """
    root_path = Path(root_dir)
    # Recursively find all JSON files
    json_files = list(root_path.rglob("*.json"))

    if not json_files:
        print(f"No JSON files found in {root_dir}")
        return pd.DataFrame()

    all_dfs = []
    for f in json_files:
        try:
            rows = extract_experiment_data(f)
            all_dfs.extend(rows)
        except Exception as e:
            print(f"Failed to extract data from {f}")
            raise e

    if not all_dfs:
        print("No valid data extracted from found files.")
        return pd.DataFrame()

    # Concatenate all individual DataFrames by row
    final_df = pd.DataFrame.from_records(all_dfs)

    return final_df

df = aggregate_experiment_results('./results/')

## FFT, KD-LoRA, LoRA

In [83]:
TOTAL_RANKS = [15, 31]
MODEL_FAMILY = 'bert'

In [84]:
df = df[(df.variant == 'fft') | (df.peft.str.contains('mrlora') & df['rank'].isin(TOTAL_RANKS))]
df = df[df.family == MODEL_FAMILY]

In [85]:
for key, value in METRIC_NAME_MAP.items():
    df.replace(key, value, inplace=True)
for key, value in TASK_NAME_MAP.items():
    df.replace(key, value, inplace=True)

In [86]:
df['value'] = df.value * 100

In [87]:
df['rank'].unique(), df.family.unique(), df.peft.unique(), df.task.unique(), df.metric.unique()

(array([15, 31,  8]),
 array(['bert'], dtype=object),
 array(['mrlora', 'mrlora-rs', 'lora'], dtype=object),
 array(['wnli', 'CoLA', 'SST-2', 'MRPC', 'MNLI', 'QNLI', 'STS-B', 'QQP',
        'RTE'], dtype=object),
 array(['Acc', 'Mcc', 'F1', 'm', 'mm', 'Corr_p', 'Corr_s'], dtype=object))

In [88]:
# 1. 格式化 params 的函数
def format_params(x):
    val = float(x)
    # 如果是整数（如 184.0），显示为 184M
    if val.is_integer():
        return f"{int(val)}M"
    # 如果有小数（如 0.312），保留两位显示为 0.31M
    else:
        return f"{val:.2f}M"


In [89]:
import pandas as pd

# 1. Define the Method Label logic
def get_method_name(row):
    if row['variant'] == 'fft':
        return 'Full FT'
    elif row['variant'] == 'lora':
        return f"LoRA$_{{r={int(row['rank'])}}}$"
    elif row['variant'] == 'kd-lora':
        return f"KD-LoRA$_{{r={int(row['rank'])}}}$"
    return row['variant']

df['Method'] = df.apply(get_method_name, axis=1)
df['params'] = df.groupby(['variant', 'rank'])['params'].transform('first')
df['params_formatted'] = df['params'].apply(format_params)

# 2. Combine multi-metric tasks (MNLI m/mm and QQP Acc/F1)
# We create a helper function to merge values into strings
def format_values(group):
    task = group['task'].iloc[0]
    if task == 'MNLI':
        # Assumes 'm' and 'mm' metrics exist for MNLI
        m = group[group['metric'] == 'm']['value'].iloc[0]
        mm = group[group['metric'] == 'mm']['value'].iloc[0]
        return pd.Series({'val': f"{m:.2f}/{mm:.2f}", 'met': 'm/mm'})
    elif task == 'QQP':
        # Assumes 'Acc' and 'F1' metrics exist for QQP
        acc = group[group['metric'] == 'Acc']['value'].iloc[0]
        f1 = group[group['metric'] == 'F1']['value'].iloc[0]
        return pd.Series({'val': f"{acc:.2f}/{f1:.2f}", 'met': 'Acc/F1'})
    elif task == 'STS-B':
        corr_s = group[group['metric'] == 'Corr_s']['value'].iloc[0]
        corr_p = group[group['metric'] == 'Corr_p']['value'].iloc[0]
        return pd.Series({'val': f"{corr_p:.2f}/{corr_s:.2f}", 'met': 'Pear/Spear'})
    else:
        # Standard tasks with single metrics
        return pd.Series({'val': f"{group['value'].iloc[0]:.2f}", 'met': group['metric'].iloc[0]})

df_transformed = df.groupby(['variant', 'rank', 'Method', 'params_formatted', 'task'], as_index=False).apply(format_values)

# 3. Pivot the table
# Index: Method and Params
# Columns: Task and the combined Metric name
pivot_df = df_transformed.pivot(
    index=['variant', 'rank', 'Method', 'params_formatted'],
    columns=['task', 'met'],
    values='val'
)

# 4. Custom Sorting
# Use variant to put 'fft' first, then rank descending (31 then 15)
pivot_df = pivot_df.sort_index(level=['variant', 'rank'], ascending=[True, False])

# Clean up index: remove the helper columns used for sorting
pivot_df.index = pivot_df.index.droplevel(['variant', 'rank'])

# 5. Column Ordering (to match the image)
task_order = ['MNLI', 'SST-2', 'CoLA', 'QQP', 'QNLI', 'RTE', 'MRPC', 'STS-B', 'All']
# Filter tasks to only those present in your data
existing_tasks = [t for t in task_order if t in pivot_df.columns.get_level_values(0)]
pivot_df = pivot_df.reindex(columns=existing_tasks, level=0)

pivot_df.index.names = ['Method', '\# Params']

# Display result
print(pivot_df)

task                               MNLI  SST-2   CoLA          QQP   QNLI    RTE   MRPC        STS-B
met                                m/mm    Acc    Mcc       Acc/F1    Acc    Acc    Acc   Pear/Spear
Method           \# Params                                                                          
Full FT          109.48M    83.27/83.58  92.55  55.21  90.93/88.01  90.74  64.98  87.01  88.26/87.79
KD-LoRA$_{r=31}$ 1.16M      80.02/80.44  89.68  44.28  87.72/83.40  86.53  55.23  71.81  83.46/83.21
KD-LoRA$_{r=15}$ 0.87M      79.24/80.04  90.60  42.10  87.44/82.96  86.14  53.07  72.55  83.32/83.26
LoRA$_{r=31}$    1.14M      82.80/83.43  91.28  50.23  89.07/85.27  90.44  55.96  78.92  87.78/87.62
LoRA$_{r=15}$    0.55M      83.16/83.54  91.51  48.60  88.64/84.63  90.26  56.32  79.66  87.32/87.12


<>:63: SyntaxWarning: invalid escape sequence '\#'
<>:63: SyntaxWarning: invalid escape sequence '\#'
/var/folders/82/tr0t3jp906z8k_q9pbcg_jd40000gn/T/ipykernel_74239/847689742.py:63: SyntaxWarning: invalid escape sequence '\#'
  pivot_df.index.names = ['Method', '\# Params']
/var/folders/82/tr0t3jp906z8k_q9pbcg_jd40000gn/T/ipykernel_74239/847689742.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_transformed = df.groupby(['variant', 'rank', 'Method', 'params_formatted', 'task'], as_index=False).apply(format_values)


In [92]:
pivot_df.columns.names = [None, None]
pivot_df.index.names = [None, None]

In [93]:
pivot_df

,,MNLI,SST-2,CoLA,QQP,QNLI,RTE,MRPC,STS-B
,,m/mm,Acc,Mcc,Acc/F1,Acc,Acc,Acc,Pear/Spear
Full FT,109.48M,83.27/83.58,92.55,55.21,90.93/88.01,90.74,64.98,87.01,88.26/87.79
KD-LoRA$_{r=31}$,1.16M,80.02/80.44,89.68,44.28,87.72/83.40,86.53,55.23,71.81,83.46/83.21
KD-LoRA$_{r=15}$,0.87M,79.24/80.04,90.60,42.10,87.44/82.96,86.14,53.07,72.55,83.32/83.26
LoRA$_{r=31}$,1.14M,82.80/83.43,91.28,50.23,89.07/85.27,90.44,55.96,78.92,87.78/87.62
LoRA$_{r=15}$,0.55M,83.16/83.54,91.51,48.60,88.64/84.63,90.26,56.32,79.66,87.32/87.12


In [95]:
pivot_df.columns

MultiIndex([( 'MNLI',       'm/mm'),
            ('SST-2',        'Acc'),
            ( 'CoLA',        'Mcc'),
            (  'QQP',     'Acc/F1'),
            ( 'QNLI',        'Acc'),
            (  'RTE',        'Acc'),
            ( 'MRPC',        'Acc'),
            ('STS-B', 'Pear/Spear')],
           )

In [96]:
# 2. Use Styler to generate the LaTeX code
latex_code = pivot_df.style.to_latex(
    column_format='l|c|' + 'c' * len(pivot_df.columns),
    hrules=True,
    multicol_align="c",
    multirow_align="c"
)

# 3. Adjust spacing for the 'tight' look in the image
final_latex = (
    "\\begin{table}[h]\n"
    "\\centering\n"
    "\\setlength{\\tabcolsep}{4pt} % Smaller column gap\n"
    "\\renewcommand{\\arraystretch}{1.2} % Better vertical spacing\n"
    f"{latex_code}\n"
    "\\end{table}"
)

print(final_latex)

\begin{table}[h]
\centering
\setlength{\tabcolsep}{4pt} % Smaller column gap
\renewcommand{\arraystretch}{1.2} % Better vertical spacing
\begin{tabular}{l|c|cccccccc}
\toprule
 &  & MNLI & SST-2 & CoLA & QQP & QNLI & RTE & MRPC & STS-B \\
 &  & m/mm & Acc & Mcc & Acc/F1 & Acc & Acc & Acc & Pear/Spear \\
\midrule
Full FT & 109.48M & 83.27/83.58 & 92.55 & 55.21 & 90.93/88.01 & 90.74 & 64.98 & 87.01 & 88.26/87.79 \\
KD-LoRA$_{r=31}$ & 1.16M & 80.02/80.44 & 89.68 & 44.28 & 87.72/83.40 & 86.53 & 55.23 & 71.81 & 83.46/83.21 \\
KD-LoRA$_{r=15}$ & 0.87M & 79.24/80.04 & 90.60 & 42.10 & 87.44/82.96 & 86.14 & 53.07 & 72.55 & 83.32/83.26 \\
LoRA$_{r=31}$ & 1.14M & 82.80/83.43 & 91.28 & 50.23 & 89.07/85.27 & 90.44 & 55.96 & 78.92 & 87.78/87.62 \\
LoRA$_{r=15}$ & 0.55M & 83.16/83.54 & 91.51 & 48.60 & 88.64/84.63 & 90.26 & 56.32 & 79.66 & 87.32/87.12 \\
\bottomrule
\end{tabular}

\end{table}


In [ ]:
# 生成 LaTeX 代码
latex_code = pivot_df.to_latex(
    # multicol_aggregator='center', 
    column_format='l|c|' + 'c'*len(pivot_df.columns),
    caption="",
    label=""
)
latex_code = latex_code.replace('task', '').replace('met', '')
print(latex_code)

\begin{tabular}{l|c|cccccccc}
\toprule
 &  & MNLI & SST-2 & CoLA & QQP & QNLI & RTE & MRPC & STS-B \\
 &  & m/mm & Acc & Mcc & Acc/F1 & Acc & Acc & Acc & Pear/Spear \\
Method & \# Params &  &  &  &  &  &  &  &  \\
\midrule
Full FT & 109.48M & 83.27/83.58 & 92.55 & 55.21 & 90.93/88.01 & 90.74 & 64.98 & 87.01 & 88.26/87.79 \\
\cline{1-10}
KD-LoRA$_{r=31}$ & 1.16M & 80.02/80.44 & 89.68 & 44.28 & 87.72/83.40 & 86.53 & 55.23 & 71.81 & 83.46/83.21 \\
\cline{1-10}
KD-LoRA$_{r=15}$ & 0.87M & 79.24/80.04 & 90.60 & 42.10 & 87.44/82.96 & 86.14 & 53.07 & 72.55 & 83.32/83.26 \\
\cline{1-10}
LoRA$_{r=31}$ & 1.14M & 82.80/83.43 & 91.28 & 50.23 & 89.07/85.27 & 90.44 & 55.96 & 78.92 & 87.78/87.62 \\
\cline{1-10}
LoRA$_{r=15}$ & 0.55M & 83.16/83.54 & 91.51 & 48.60 & 88.64/84.63 & 90.26 & 56.32 & 79.66 & 87.32/87.12 \\
\cline{1-10}
\bottomrule
\end{tabular}

